# High Glucose Scenario

## Connect to BioExplorer backend

In [ ]:
from bioexplorer import BioExplorer, RNASequence, Protein, AssemblyProtein, Virus, Surfactant, Membrane, Cell, Sugars, \
    Volume, Mesh, Vector2, Vector3, Quaternion
import nglview

hostname = 'localhost:5000'
resource_folder = '../tests/test_files/'

be = BioExplorer(hostname)
be.version()
be.reset()

export_to_cache = True

## Global parameters

In [ ]:
# Scene
scene_size = 800.0

# Glycans
add_glycans = True
glycan_radius_multiplier = 1.0
glycan_add_sticks = True

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS
protein_load_hydrogen = False

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = True

# Immune system
nb_glucoses = 360000
nb_lactoferrins = 50
nb_defensins = 100
nb_defensins_on_virus = 2

## Resources

In [ ]:
cache_folder = resource_folder + 'caches/0.6.0/'
image_folder = resource_folder + 'images/'
pdb_folder = resource_folder + 'pdb/'
rna_folder = resource_folder + 'rna/'
obj_folder = resource_folder + 'obj/'
glycan_folder = pdb_folder + 'glycans/'

complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

## Coronaviruses

In [ ]:
def add_virus(name, position, open_conformation_indices=list()):
    
    closed_conformation_indices = list()
    for i in range(nb_protein_s):
        if i not in open_conformation_indices:
            closed_conformation_indices.append(i)

    virus_protein_s = Protein(
        sources=[
            pdb_folder + '6vyb.pdb',  # Open conformation
            pdb_folder + 'sars-cov-2-v1.pdb'  # Closed conformation
        ],
        load_hydrogen=protein_load_hydrogen, number_of_instances=nb_protein_s, 
        assembly_params=Vector2(11.5, 0.0), cutoff_angle=0.999,
        orientation=Quaternion(0.087, 0.0, 0.996, 0.0),
        instance_indices=[open_conformation_indices, closed_conformation_indices])

    virus_protein_m = Protein(
        sources=[pdb_folder + 'QHD43419a.pdb'], load_hydrogen=protein_load_hydrogen,
        number_of_instances=nb_protein_m, assembly_params=Vector2(2.0, 0.0),
        cutoff_angle=0.999, orientation=Quaternion(0.99, 0.0, 0.0, 0.135))

    virus_protein_e = Protein(
        sources=[pdb_folder + 'QHD43418a.pdb'], load_hydrogen=protein_load_hydrogen, 
        number_of_instances=nb_protein_e, assembly_params=Vector2(3.0, 0.0), 
        cutoff_angle=0.9999, orientation=Quaternion(0.705, 0.705, -0.04, -0.04))

    virus_membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'],
        number_of_instances=15000)

    rna_sequence = None
    clip_planes = list()
    if add_rna:
        clip_planes.append([0, 0, -1, 15])
        import math
        rna_sequence = RNASequence(
            source=rna_folder + 'sars-cov-2.rna',
            assembly_params=Vector2(11.0, 0.5),
            t_range=Vector2(0, 30.5 * math.pi), shape=be.RNA_SHAPE_TREFOIL_KNOT,
            shape_params=Vector3(1.51, 1.12, 1.93))

    coronavirus = Virus(
        name=name, protein_s=virus_protein_s, protein_e=virus_protein_e,
        protein_m=virus_protein_m, membrane=virus_membrane, rna_sequence=rna_sequence,
        assembly_params=Vector2(45.0, 1.5))

    be.add_virus(
        virus=coronavirus, position=position, representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier, clipping_planes=clip_planes)

    if add_glycans:
        # High-mannose
        indices = [61, 122, 234, 603, 709, 717, 801, 1074]
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=high_mannose_paths, 
            indices=indices, add_sticks=glycan_add_sticks, 
            allowed_occurrences=closed_conformation_indices)
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=high_mannose_paths,
            indices=indices, index_offset=19, add_sticks=glycan_add_sticks,
            allowed_occurrences=open_conformation_indices)

        # Complex
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=complex_paths, 
            indices=[17, 74, 149, 165, 282, 331, 343, 616, 1098, 1134, 1158, 1173, 1194],
            add_sticks=glycan_add_sticks, allowed_occurrences=closed_conformation_indices)
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=complex_paths, 
            indices=[17, 74, 149, 165, 282, 331, 343, 1098, 1134, 1158, 1173, 1194],
            index_offset=19, add_sticks=glycan_add_sticks,
            allowed_occurrences=open_conformation_indices)

        # Hybrid
        indices = [657]
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HYBRID, 
            protein_name=be.NAME_PROTEIN_S_CLOSED, paths=hybrid_paths,
            indices=indices, add_sticks=glycan_add_sticks,
            allowed_occurrences=closed_conformation_indices)
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HYBRID, 
            protein_name=be.NAME_PROTEIN_S_OPEN, paths=hybrid_paths, 
            indices=indices, index_offset=19, add_sticks=glycan_add_sticks,
            allowed_occurrences=open_conformation_indices)

        # O-Glycans
        for index in [323, 325]:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index)
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_PROTEIN_S_CLOSED, 
                add_sticks=glycan_add_sticks, site_indices=[index])
            be.add_sugars(o_glycan)

        # High-mannose glycans on Protein M
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_HIGH_MANNOSE,
            protein_name=be.NAME_PROTEIN_M,
            paths=high_mannose_paths, add_sticks=glycan_add_sticks)

        # Complex glycans on Protein E
        be.add_multiple_glycans(
            assembly_name=name, glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_PROTEIN_E, paths=complex_paths,
            add_sticks=glycan_add_sticks)
        
    # Stick defensins to virus
    for i in range(nb_defensins_on_virus):
        defensin = AssemblyProtein(
            assembly_name=name, name=name + '_' + be.NAME_DEFENSIN + '_' + str(i),
            source=defensin_path, assembly_params=Vector2(2.0, 0.0), 
            shape=be.ASSEMBLY_SHAPE_SPHERICAL,
            atom_radius_multiplier=protein_radius_multiplier, 
            representation=protein_representation,
            random_seed=100 + i + 1)
        be.add_assembly_protein(defensin)

In [ ]:
add_virus(name='Coronavirus 1', position=Vector3(-280.5, -97.0, -82.5), 
          open_conformation_indices=[1])
add_virus(name='Coronavirus 2', position=Vector3( -70.5, -102.0, 229.5),
          open_conformation_indices=[1])
add_virus(name='Coronavirus 3', position=Vector3(296.5, -125.0, 225.5),
          open_conformation_indices=[1])
add_virus(name='Coronavirus 4', position=Vector3(4.5, 100.0, 7.5),
          open_conformation_indices=[1])
add_virus(name='Coronavirus 5', position=Vector3(204.5, -100.0, 27.5), 
          open_conformation_indices=[1])
add_virus(name='Coronavirus 6', position=Vector3(54.5, -100.0, -242.5),
          open_conformation_indices=[1])

In [ ]:
nglview.show_file(resource_folder + 'pdb/sars-cov-2-v1.pdb')

In [ ]:
nglview.show_file(resource_folder + 'pdb/QHD43419a.pdb')

In [ ]:
nglview.show_file(resource_folder + 'pdb/QHD43418a.pdb')

In [ ]:
if export_to_cache:
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    cache_filename = cache_folder + 'high_glucose_viruses.bioexplorer'
    if add_rna:
        cache_filename = cache_folder + 'high_glucose_open_viruses.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)

## Host cell

In [ ]:
def add_cell(name, size, height, nb_receptors, position=Vector3()):
    ace2_receptor = Protein(
        sources=[pdb_folder + '6m1d.pdb'], number_of_instances=nb_receptors,
        position=Vector3(0.0, 6.0, 0.0))
    
    membrane = Membrane(
        sources=[pdb_folder + 'membrane/popc.pdb'], number_of_instances=1200000)

    cell = Cell(
        name=name, size=Vector2(size, height), shape=be.ASSEMBLY_SHAPE_SINUSOIDAL, 
        membrane=membrane, receptor=ace2_receptor)
    
    be.add_cell(cell=cell, position=position, representation=protein_representation)

    if add_glycans:
        be.add_multiple_glycans(
            add_sticks=glycan_add_sticks, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[62, 99, 112, 331, 441, 699])
        be.add_multiple_glycans(
            add_sticks=glycan_add_sticks, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[555])

        indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugars(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, add_sticks=glycan_add_sticks,
                chain_ids=[2, 4], site_indices=[index[0]], orientation=index[1])
            be.add_sugars(o_glycan)

In [ ]:
add_cell(name='Cell 1',
         size=scene_size, 
         height=scene_size / 10.0,
         nb_receptors=20,
         position=Vector3(4.5, -186.0, 7.0))

In [ ]:
if export_to_cache:
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    be.set_protein_color_scheme(
        'Cell 1', 'Cell 1' + '_' + be.NAME_RECEPTOR, be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)
    cache_filename = cache_folder + 'cell.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)

## Immune system

### Surfactant A and D

In [ ]:
nglview.show_file(surfactant_head_source)

In [ ]:
def add_surfactant_d(name, position, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, representation=protein_representation, 
        position=position, random_seed=random_seed)

add_surfactant_d(
    name='Surfactant-D 1', position=Vector3(74.0, 24.0, -45.0), random_seed=1)
add_surfactant_d(
    name='Surfactant-D 2', position=Vector3(104.0, 175.0, -89.0), random_seed=2)
add_surfactant_d(
    name='Surfactant-D 3', position=Vector3(-260.0, 50.0, 0.0), random_seed=6)

In [ ]:
def add_surfactant_a(name, position, random_seed):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, representation=protein_representation, 
        position=position, random_seed=random_seed)

add_surfactant_a(
    name='Surfactant-A 1', position=Vector3(-100.0, 150.0, 0.0), random_seed=2)

In [ ]:
# Add glucose to surfactants
def add_glucose_to_surfactant_head(name):
    for index in [321, 323]:
        glucose_name = name + '_' + be.NAME_GLUCOSE + '_' + str(index)
        glucose = Sugars(
            assembly_name=name, name=glucose_name, source=glucose_path,
            protein_name=name + '_' + be.NAME_SURFACTANT_HEAD, 
            add_sticks=glycan_add_sticks, site_indices=[index])
        be.add_sugars(glucose)
add_glucose_to_surfactant_head('Surfactant-D 1')
add_glucose_to_surfactant_head('Surfactant-D 2')
add_glucose_to_surfactant_head('Surfactant-D 3')
add_glucose_to_surfactant_head('Surfactant-A 1')

### Glucose

In [ ]:
glucose = Protein(
    sources=[glucose_path], load_non_polymer_chemicals=True, 
    number_of_instances=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, size=Vector2(scene_size, scene_size), protein=glucose)
be.add_volume(
    volume=volume, representation=be.REPRESENTATION_ATOMS,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0))

In [ ]:
nglview.show_file(glucose_path)

### Lactoferrins

In [ ]:
lactoferrin = Protein(
    sources=[lactoferrin_path], load_non_polymer_chemicals=True,
    number_of_instances=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, size=Vector2(scene_size, scene_size), protein=lactoferrin)
be.add_volume(
    volume=lactoferrins_volume, representation=be.REPRESENTATION_ATOMS,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0))

In [ ]:
nglview.show_file(lactoferrin_path)

### Defensins

In [ ]:
defensin = Protein(
    sources=[defensin_path], load_non_polymer_chemicals=True, 
    number_of_instances=nb_defensins)
defensins_volume = Volume(
    name=be.NAME_DEFENSIN, size=Vector2(scene_size, scene_size), protein=defensin)
be.add_volume(
    volume=defensins_volume, representation=be.REPRESENTATION_ATOMS,
    position=Vector3(0.0, scene_size / 2.0 - 200.0, 0.0))

In [ ]:
nglview.show_file(defensin_path)

In [ ]:
if export_to_cache:
#     be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    cache_filename = cache_folder + 'high_glucose_immune.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)

## Materials

In [ ]:
be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)

In [ ]:
for model in be.core_api().scene.models:
    model_id = model['id']
    model_name = model['name']
    if be.NAME_COLLAGEN in model_name:
        material_ids = list(be.get_material_ids(model_id)['ids'])
        nb_materials = len(material_ids)
        print(model_name)
        palette = list()
        emissions = list()
        for i in range(nb_materials):
            palette.append([1,1,1])
            emissions.append(0.1)
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids,
            diffuse_colors=palette, specular_colors=palette,
            emissions=emissions
        )        

In [ ]:
if export_to_cache:
    cache_filename = cache_folder + 'high_glucose_scenario.bioexplorer'
    print(cache_filename)
    be.export_to_cache(cache_filename)

## Rendering settings

In [ ]:
be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=1, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.01
params.gi_distance = 500
params.shadows = 0.75
params.soft_shadows = 1
params.fog_start = 1500
params.fog_thickness = 1500
params.epsilon_factor = 1
params.max_bounces = 3
be.core_api().set_renderer_params(params)